In [1]:
# 필요 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import itertools
import pandas as pd
import dask.dataframe as dd
from dask import delayed

In [2]:
# Microsoft Edge WebDriver 경로
edge_driver_path = "C:/Users/yuhan/Downloads/edgedriver_win64/msedgedriver.exe"

edge_options = Options()
# edge_options.add_argument('--headless') # background에서 돌아가게하는 옵션 -> 직접 페이지를 켜두고 확인해야할 부분이 있으면 해당 옵션 주석처리
# 머신보호용으로 샌드박스를 사용하는데 일회용 컨테이너 안에 있는 거면 sandbox 불필요함
edge_options.add_argument('--no-sandbox')
# 공유 메모리 고갈로 인한 크래쉬 방지용
edge_options.add_argument('--disable-dev-shm-usage')

In [3]:
# 글로벌 변수 정의
global logged_in
logged_in = False

# Edge WebDriver 객체 생성
global driver
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

In [4]:
def login_to_glo_bus(id='won103203@naver.com', pw='goTeamC@1'):
    global logged_in
    global driver

    # Glo-Bus 로그인 페이지 접속
    driver.get('https://www.glo-bus.com/')
    driver.implicitly_wait(5)

    # 로그인 페이지에서 로그인 정보 입력
    driver.find_element(By.ID, 'acct_name').send_keys(id)
    driver.find_element(By.ID, 'passwdInput').send_keys(pw)

    # 로그인 버튼 클릭
    driver.find_element(By.ID, 'loginbutton').click()

    # 로그인 완료
    logged_in = True
    print('로그인 완료')

def open_DnR(subsection = None):
    # 로그인이 되어있지 않다면 로그인
    if not logged_in:
        login_to_glo_bus()

    # subsection이 None이 아니라면 해당 subsection으로 이동
    if subsection:
        driver.get(f'https://www.glo-bus.com/users/program21/decisions/{subsection}')
        print(f'{subsection} 페이지로 이동완료')
    else:
        # Decision and Reports 페이지로 이동
        driver.get('https://www.glo-bus.com/users/program21/')
        print('Decision and Reports 페이지로 이동완료')

In [5]:
open_DnR('product-design')

로그인 완료
product-design 페이지로 이동완료


In [ ]:
action_capture_camera_design_dict = {
    ('Image Sensor Size', 'G4-btn'): ['8mm', '9mm', '10mm', '11mm', '12mm', '13mm', '14mm'],
    ('LCD Display Screen', 'G5-btn'): ['230k', '460k', '610k', '920k', '1040k', '1230k', '2360k'],
    ('Image Quality', 'G6-btn'): ['1920×1080', '1920×1440', '2704×1520', '2704×2028', '3840×2160', '3840×2400', '4096×2160'],
    ('Number of Photo Modes', 'G7-btn'): ['4 / 3', '6 / 3', '7 / 3', '8 / 3', '10 / 4', '12 / 4', '16 / 4'],
    ('Camera Housing', 'G9-btn'): ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    ('Editing / Sharing Capabilities', 'G10-btn'): ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    ('Included Accessories', 'G11-btn'): ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'],
    ('Extra Performance Features', 'G13-btn'): ['2', '3', '4', '5', '6', '7', '8', '9', '10'],
    ('Number of Action Capture Camera Models', 'G14-btn'): ['1', '2', '3', '4', '5', '6', '7'],
    ('Product R&D Expenditures', 'G16-btn'): ['0', '3000', '6000', '9000', '12000', '16000', '20000', '24000', '28000', '32000', '36000', '40000', '45000', '50000']
}

uav_drone_design_dict = {
    ('Built-In Camera', 'G22-btn'): ['No Upgrade', 'Minor Upgrade', 'Significant Upgrade', 'Major Upgrade'],
    ('GPS / WiFi / Bluetooth', 'G23-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Battery Pack', 'G24-btn'): ['8', '10', '12', '15', '18', '21', '25', '30'],
    ('Number of Rotors', 'G25-btn'): ['4', '6', '8'],
    ('Rotor Performance / Flight Controller', 'G26-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Body Frame Construction', 'G27-btn'): ['High-Strength Plastic', 'g10/FR4 Fiberglass', 'Carbon Fiber'],
    ('Obstacle Sensors', 'G28-btn'): ['Front Only', 'Front/Rear', '360° Basic', '360° Enhanced', '360° Advanced', '360° Best Available'],
    ('Camera Stabilization Device', 'G29-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Extra Performance Features', 'G31-btn'): ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    ('Number of UAV Drone Models', 'G32-btn'): ['1', '2', '3', '4', '5', '6', '7'],
    ('Product R&D Expenditures', 'G34-btn'): ['0', '3000', '6000', '9000', '12000', '15000', '18000', '21000', '24000', '27000', '30000', '35000', '40000', '45000', '50000']
}

In [ ]:
def scrape_PD_results(item):
    if item == 'action_camera':
        # pq_rating_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(1) > div.card.mb-2.ac-calc-area1.perf-quality > div > table > tbody > tr > td.align-middle > div > span:nth-child(1) > strong'
        # pa_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(1) > div.card.mb-2.ac-calc-area2 > div > table > tbody > tr:nth-child(15) > td:nth-child(3) > div > span:nth-child(2) > strong'
        pq_rating_xpath = '/html/body/glo-app/section/div[2]/div/glo-decisions/div/div[1]/glo-dec-product-design/div/div[1]/div[3]/div/table/tbody/tr/td[2]/div/span[1]/strong'
        pa_cost_per_unit_xpath = '/html/body/glo-app/section/div[2]/div/glo-decisions/div/div[1]/glo-dec-product-design/div/div[1]/div[4]/div/table/tbody/tr[15]/td[3]/div/span[2]/strong'
    elif item == 'drone':
        # pq_rating_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(2) > div.card.mb-2.uav-calc-area1.perf-quality > div > table > tbody > tr > td.align-middle > div > span:nth-child(1) > strong'
        # pa_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(2) > div.card.mb-2.uav-calc-area2 > div > table > tbody > tr:nth-child(15) > td:nth-child(3) > div > span:nth-child(2) > strong'
        pq_rating_xpath = '/html/body/glo-app/section/div/div[2]/glo-decisions/div/div[1]/glo-dec-product-design/div/div[2]/div[3]/div/table/tbody/tr/td[2]/div/span[1]/strong'
        pa_cost_per_unit_xpath = '/html/body/glo-app/section/div/div[2]/glo-decisions/div/div[1]/glo-dec-product-design/div/div[2]/div[4]/div/table/tbody/tr[15]/td[3]/div/span[2]/strong'
    
    # Wait for the page to load
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, pq_rating_css)))
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, pa_cost_per_unit_css)))
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, pq_rating_xpath)))
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, pa_cost_per_unit_xpath)))


    # pq_rating = driver.find_element(By.CSS_SELECTOR, pq_rating_css).text
    # pa_cost_per_unit = driver.find_element(By.CSS_SELECTOR, pa_cost_per_unit_css).text
    pq_rating = driver.find_element(By.XPATH, pq_rating_xpath).text
    pa_cost_per_unit = driver.find_element(By.XPATH, pa_cost_per_unit_xpath).text

    return pq_rating, pa_cost_per_unit

In [ ]:
def fix_design_choices(design_dict, fix_values_dict):
    """
    design_dict: 디자인 선택지 딕셔너리
    fix_values_dict: 고정할 선택지 딕셔너리
    design_dict에서 fix_values_dict의 key에 해당하는 선택지를 values로 고정
    새로운 선택지 딕셔너리를 반환
    """
    new_design_dict = design_dict.copy()
    for key, value in fix_values_dict.items():
        new_design_dict[key] = [value]
    return new_design_dict

In [ ]:
def calculate_number_of_design_options(design_dict):
    """
    Returns the number of design options for a given design dictionary.
    """

    # Calculate the number of design options
    num_design_options = 1
    for key in design_dict.keys():
        num_design_options *= len(design_dict[key])

    return num_design_options

print(calculate_number_of_design_options(action_capture_camera_design_dict))
print(calculate_number_of_design_options(uav_drone_design_dict))

In [8]:
def generate_chunks(value_lists, columns, chunk_size):
    """
    Generator function that yields DataFrame chunks of the Cartesian product.
    """
    # Create an iterator for all possible combinations
    iterator = itertools.product(*value_lists)
    while True:
        # Grab the next chunk_size combinations
        chunk = list(itertools.islice(iterator, chunk_size))
        if not chunk:
            break
        yield pd.DataFrame(chunk, columns=columns)

def generate_design_combinations(product, design_dir_name, design_dict, chunk_size=1000000):
    """
    Returns a list of dictionaries, where each dictionary represents a unique design combination.
    """
    # Extract column names (first element of each key tuple) and value lists
    columns = [key[1] for key in design_dict.keys()]
    value_lists = list(design_dict.values())

    # Wrap each chunk with dask.delayed so Dask can build a lazy graph
    delayed_dfs = [delayed(chunk_df) for chunk_df in generate_chunks(value_lists, columns, chunk_size)]

    # Create a Dask DataFrame from the delayed chunks
    ddf = dd.from_delayed(delayed_dfs)

    # Write the Dask DataFrame to a Parquet file.
    # This will write the data in partitions (chunks) to the '{product}/{design_dir_name}.parquet' directory.
    ddf.to_parquet(f'product_design_query/{product}/{design_dir_name}.parquet', write_index=False)

    print(f'Finished generating design combinations for {product} using {design_dir_name}.')

In [ ]:
generate_design_combinations('drone', 'all_combinations', uav_drone_design_dict)

In [9]:
# Read in the Parquet file as a Dask DataFrame
ddf = dd.read_parquet('product_design_query/drone/all_combinations.parquet')

# Choose only first 100 rows for testing
ddf_test = ddf.head(100)

In [26]:
# Use selenium to change parameters in the product design page
# Each set of parameters is a row in the DataFrame
# For each row, we will:
# 1. Set the parameters
# 2. Read the PQ Rating and PA Cost per Unit
# 3. Save the results to a new column in the datafram

def set_design_parameters(df):
    for i, row in df.iterrows():
        # Set the design parameters
        for key, value in row.items():
            
            # Update the design parameters
            element = driver.find_element(By.ID, key)
            select = Select(element)
            select.select_by_visible_text(value)

        # Scrape the results
        pq_rating, pa_cost_per_unit = scrape_PD_results('drone')
        print(f'PQ Rating: {pq_rating}, PA Cost per Unit: {pa_cost_per_unit}')

set_design_parameters(ddf)

PQ Rating: 1.5, PA Cost per Unit: 584.80
PQ Rating: 1.5, PA Cost per Unit: 618.81
PQ Rating: 1.5, PA Cost per Unit: 652.82
PQ Rating: 1.5, PA Cost per Unit: 686.84
PQ Rating: 1.5, PA Cost per Unit: 720.85
PQ Rating: 1.5, PA Cost per Unit: 754.86
PQ Rating: 1.5, PA Cost per Unit: 788.88
PQ Rating: 1.5, PA Cost per Unit: 822.89
PQ Rating: 1.5, PA Cost per Unit: 856.90
PQ Rating: 1.5, PA Cost per Unit: 889.84
PQ Rating: 1.5, PA Cost per Unit: 923.85
PQ Rating: 1.5, PA Cost per Unit: 980.54
PQ Rating: 1.5, PA Cost per Unit: 1,037.23
PQ Rating: 1.6, PA Cost per Unit: 1,065.80
PQ Rating: 1.6, PA Cost per Unit: 1,119.71
PQ Rating: 1.2, PA Cost per Unit: 587.82
PQ Rating: 1.2, PA Cost per Unit: 620.83
PQ Rating: 1.2, PA Cost per Unit: 653.83
PQ Rating: 1.2, PA Cost per Unit: 686.83
PQ Rating: 1.2, PA Cost per Unit: 719.83
PQ Rating: 1.2, PA Cost per Unit: 752.84
PQ Rating: 1.2, PA Cost per Unit: 785.84
PQ Rating: 1.2, PA Cost per Unit: 818.84
PQ Rating: 1.2, PA Cost per Unit: 851.85
PQ Rating:

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C3AEE735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7C3D7B1D4+2060308]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7C3CD2338+1368440]
	(No symbol) [0x00007FF7C38C32A9]
	(No symbol) [0x00007FF7C38C349B]
	(No symbol) [0x00007FF7C39040E7]
	(No symbol) [0x00007FF7C38E429F]
	(No symbol) [0x00007FF7C38B964D]
	(No symbol) [0x00007FF7C3901C00]
	(No symbol) [0x00007FF7C38E3FC3]
	(No symbol) [0x00007FF7C38B8B96]
	(No symbol) [0x00007FF7C38B7E50]
	(No symbol) [0x00007FF7C38B89C3]
	(No symbol) [0x00007FF7C3940214]
	(No symbol) [0x00007FF7C39D06CF]
	(No symbol) [0x00007FF7C3943A53]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7C3E345DD+279981]
	simdutf::get_active_implementation [0x00007FF7C3A778C1+409425]
	simdutf::get_active_implementation [0x00007FF7C3A72BA4+389684]
	simdutf::get_active_implementation [0x00007FF7C3A72CE9+390009]
	simdutf::get_active_impl

In [ ]:
# Use selenium to change parameters in the product design page
# Each set of parameters is a row in the DataFrame
# For each row, we will:
# 1. Set the parameters
# 2. Read the PQ Rating and PA Cost per Unit
# 3. Save the results to a new column in the dataframe

def set_parameters_and_read_results(ddf_test):
    # Open the Product Design page
    open_DnR('product-design')

    # Get the row as a dictionary
    row = ddf_test.to_dict()

    # Set the parameters for the UAV Drone
    for key in uav_drone_design_dict.keys():
        element_id = key[1]
        value = row[key[1]]
        element = driver.find_element(By.ID, element_id)
        select = Select(element)
        select.select_by_visible_text(value)

    # Read the PQ Rating and PA Cost per Unit
    pq_rating, pa_cost_per_unit = scrape_PD_results('drone')

    # Add the results to a new column in the Dask DataFrame
    row['PQ Rating'] = pq_rating
    row['PA Cost per Unit'] = pa_cost_per_unit
    row['PQ Rating vs. PA Cost per Unit'] = pq_rating / pa_cost_per_unit

    return row

# Define a named function to apply the row-wise operation
def apply_set_parameters_and_read_results(df):
    return df.apply(set_parameters_and_read_results, axis=1)

# Apply the function to the test pandas DataFrame
ddf_results = ddf_test.map(apply_set_parameters_and_read_results)

# Save the results to a Parquet file
# ddf_results.to_parquet('product_design_query/drone/all_combinations_results.parquet', write_index=False)

AttributeError: 'str' object has no attribute 'apply'

In [ ]:
def click_resume_button():
    # Resume 버튼 클릭
    resume_button_xpath = '/html/body/glo-app/section/glo-keep-alive/div/div/div/div[3]/button'
    if driver.find_element(By.XPATH, resume_button_xpath).is_displayed():
        driver.find_element(By.XPATH, resume_button_xpath).click()
    else:
        pass

In [ ]:
click_resume_button()

In [ ]:
driver.quit()